## 🎯 Objectif du notebook `06_predict.ipynb`

Ce notebook a pour objectif de **charger les meilleurs modèles** sélectionnés lors des étapes précédentes et de **réaliser des prédictions** sur de nouvelles données.

### ❓ Pourquoi ne teste-t-on pas plusieurs modèles ici ?

Les modèles utilisés sont ceux ayant déjà été **rigoureusement comparés et validés** dans les notebooks précédents :

- 📘 `03_model_regression.ipynb`  
  ➤ Sélection du modèle **XGBoost** comme meilleur prédicteur de prix, basé sur les métriques MAE, RMSE, et R².

- 📘 `04_model_classification.ipynb`  
  ➤ Sélection du modèle **Random Forest** pour la classification binaire (tranche de prix), avec un score **Accuracy > 96%**.

Il ne s’agit donc **pas ici de réentraîner ni de comparer d’autres modèles**, mais de :
- charger les modèles finalisés,
- appliquer les transformations nécessaires,
- produire les prédictions finales.

Cette approche garantit :
- la **cohérence avec l’ensemble du pipeline**,
- un **gain de temps** pour les prédictions réelles,
- une **traçabilité claire** des choix méthodologiques pour la soutenance.

---

In [1]:
# 📘 06_predict.ipynb — Évaluation croisée de toutes les paires de modèles

import os
import pandas as pd
import joblib
import numpy as np
from sentence_transformers import SentenceTransformer

# 🔧 Configuration
REGRESSION_MODELS = [
    "decision_tree", "gradient_boosting", "knn_regressor",
    "linear_regression", "random_forest", "ridge", "xgboost"
]

CLASSIFICATION_MODELS = [
    "decision_tree", "knn_classifier", "logistic_regression", "random_forest"
]

MODEL_DIR_REG = "models/regression"
MODEL_DIR_CLF = "models/classification"
SCALER_PATH = "models/regression/scaler.pkl"
COLUMNS_PATH = "models/columns_used.pkl"
OUTPUT_DIR = "data/predictions_grid"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# 🔧 Chargement des objets communs
scaler = joblib.load(SCALER_PATH)
columns = joblib.load(COLUMNS_PATH)
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 🧱 Fonction de prétraitement
def preprocess_input(description: str, fiabilite: float) -> pd.DataFrame:
    emb = embedding_model.encode([description])
    emb_dict = {f"emb_{i}": emb[0][i] for i in range(384)}
    row = {**emb_dict, "Fiabilite": fiabilite}
    df = pd.DataFrame([row])
    df = df.reindex(columns=columns, fill_value=0)
    df[["Fiabilite"]] = scaler.transform(df[["Fiabilite"]])
    return df

# 🧪 Chargement des données sources
df_source = pd.read_csv("data/fiverr_cleaned_transformed.csv")

# 🔁 Boucle sur toutes les paires de modèles
for reg_name in REGRESSION_MODELS:
    reg_model = joblib.load(f"{MODEL_DIR_REG}/{reg_name}.pkl")

    for clf_name in CLASSIFICATION_MODELS:
        clf_model = joblib.load(f"{MODEL_DIR_CLF}/{clf_name}.pkl")

        df = df_source.copy()

        # 🔮 Prédictions
        df["Prix_prédit"] = df.apply(
            lambda row: round(np.expm1(reg_model.predict(preprocess_input(row["Description"], row["Fiabilite"]))[0]) * 10, 2),
            axis=1
        )
        df["Tranche_prévue"] = df.apply(
            lambda row: clf_model.predict(preprocess_input(row["Description"], row["Fiabilite"]))[0],
            axis=1
        )

        # 💾 Sauvegarde du fichier de sortie
        filename = f"fiverr_predicted_{reg_name}__{clf_name}.csv"
        df.to_csv(os.path.join(OUTPUT_DIR, filename), index=False)
        print(f"✅ Sauvegardé : {filename}")



✅ Sauvegardé : fiverr_predicted_decision_tree__decision_tree.csv
✅ Sauvegardé : fiverr_predicted_decision_tree__knn_classifier.csv
✅ Sauvegardé : fiverr_predicted_decision_tree__logistic_regression.csv
✅ Sauvegardé : fiverr_predicted_decision_tree__random_forest.csv
✅ Sauvegardé : fiverr_predicted_gradient_boosting__decision_tree.csv
✅ Sauvegardé : fiverr_predicted_gradient_boosting__knn_classifier.csv
✅ Sauvegardé : fiverr_predicted_gradient_boosting__logistic_regression.csv
✅ Sauvegardé : fiverr_predicted_gradient_boosting__random_forest.csv
✅ Sauvegardé : fiverr_predicted_knn_regressor__decision_tree.csv
✅ Sauvegardé : fiverr_predicted_knn_regressor__knn_classifier.csv
✅ Sauvegardé : fiverr_predicted_knn_regressor__logistic_regression.csv
✅ Sauvegardé : fiverr_predicted_knn_regressor__random_forest.csv
✅ Sauvegardé : fiverr_predicted_linear_regression__decision_tree.csv
✅ Sauvegardé : fiverr_predicted_linear_regression__knn_classifier.csv
✅ Sauvegardé : fiverr_predicted_linear_regre

'\n# 📘 06_predict.ipynb — Évaluation croisée de toutes les paires de modèles\n\nimport os\nimport pandas as pd\nimport joblib\nfrom sentence_transformers import SentenceTransformer\n\n# ------------------------------------------------------------\n# 🔧 Configuration\n# ------------------------------------------------------------\nREGRESSION_MODELS = [\n    "decision_tree",\n    "gradient_boosting",\n    "knn_regressor",\n    "linear_regression",\n    "random_forest",\n    "ridge",\n    "xgboost"\n]\n\nCLASSIFICATION_MODELS = [\n    "decision_tree",\n    "knn_classifier",\n    "logistic_regression",\n    "random_forest"\n]\n\nMODEL_DIR_REG = "models/regression"\nMODEL_DIR_CLF = "models/classification"\nSCALER_PATH = "models/regression/scaler.pkl"\nCOLUMNS_PATH = "models/columns_used.pkl"\nOUTPUT_DIR = "data/predictions_grid"\n\nos.makedirs(OUTPUT_DIR, exist_ok=True)\n\n# ------------------------------------------------------------\n# 🔧 Chargement des objets communs\n# -------------------

In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import pandas as pd
import numpy as np
import os

# 📂 Répertoire des fichiers de prédiction
INPUT_DIR = "data/predictions_grid"
results = []

# 🔁 Boucle sur les fichiers de prédiction
for filename in os.listdir(INPUT_DIR):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(INPUT_DIR, filename))

        # ⚠️ Ajout de la tranche réelle si manquante
        if "Tranche_réelle" not in df.columns:
            df["Tranche_réelle"] = pd.qcut(df["Prix"], q=3, labels=["Basse", "Moyenne", "Haute"])

        # 🏷️ Extraction des noms de modèles
        reg_name, clf_name = filename.replace(".csv", "").replace("fiverr_predicted_", "").split("__")

        # 📏 Calcul des métriques
        mae = mean_absolute_error(df["Prix"], df["Prix_prédit"])
        rmse = np.sqrt(mean_squared_error(df["Prix"], df["Prix_prédit"]))
        acc = accuracy_score(df["Tranche_réelle"], df["Tranche_prévue"])

        results.append({
            "Régression": reg_name,
            "Classification": clf_name,
            "MAE": round(mae, 2),
            "RMSE": round(rmse, 2),
            "Accuracy": round(acc, 4)
        })

# 📊 Affichage des résultats
df_results = pd.DataFrame(results).sort_values(by="RMSE")
print("📈 Résultats comparatifs des paires (ordonnés par RMSE) :\n")
print(df_results.to_markdown(index=False))


📈 Résultats comparatifs des paires (ordonnés par RMSE) :

| Régression        | Classification      |   MAE |   RMSE |   Accuracy |
|:------------------|:--------------------|------:|-------:|-----------:|
| ridge             | knn_classifier      | 43.67 |  47.76 |     0.5983 |
| ridge             | logistic_regression | 43.67 |  47.76 |     0.5956 |
| ridge             | random_forest       | 43.67 |  47.76 |     0.9083 |
| ridge             | decision_tree       | 43.67 |  47.76 |     0.924  |
| knn_regressor     | random_forest       | 46.48 |  53.35 |     0.9083 |
| knn_regressor     | knn_classifier      | 46.48 |  53.35 |     0.5983 |
| knn_regressor     | decision_tree       | 46.48 |  53.35 |     0.924  |
| knn_regressor     | logistic_regression | 46.48 |  53.35 |     0.5956 |
| gradient_boosting | logistic_regression | 47.22 |  54.85 |     0.5956 |
| gradient_boosting | knn_classifier      | 47.22 |  54.85 |     0.5983 |
| gradient_boosting | decision_tree       | 47.22 |  5

In [3]:
import pandas as pd
import joblib

# Chargement du modèle entraîné (doit être un modèle à base d’arbres)
reg_model = joblib.load("models/regression/decision_tree.pkl")
columns = joblib.load("models/columns_used.pkl")

# Importance des variables
importances = reg_model.feature_importances_
features = pd.Series(importances, index=columns)

# Affichage
print("Importance de la variable Fiabilite :", round(features["Fiabilite"], 4))
print("\nTop 10 features :")
print(features.sort_values(ascending=False).head(10))

Importance de la variable Fiabilite : 0.2564

Top 10 features :
Fiabilite    0.256442
emb_78       0.057714
emb_374      0.030308
emb_121      0.024944
emb_25       0.021730
emb_363      0.021541
emb_187      0.020729
emb_60       0.018737
emb_188      0.016485
emb_260      0.016224
dtype: float64
